In [13]:
# Imports and Dependencies
import pandas as pd
import math as m
pd.set_option('display.max_rows', None)
nfa = pd.read_csv("NFA_Data.csv")
# Reads in csv for table two of the UN data, using the fifth row as header.
un = pd.read_csv("UN_Data_Table_2.csv", header=4).drop([0,67,121,159,193])

C:\Users\mrjac\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# NFA Data

In [2]:
# Creating country, year, and carbon emissions dataframe for the selected years.
nfa = nfa[['country','year','carbon','record']]
nfa_data1990 = nfa[nfa['year'] == 1990]
nfa_data2000 = nfa[nfa['year'] == 2000]
nfa_data2010 = nfa[nfa['year'] == 2010]

# Creating combined dataset.
nfa = ((nfa_data1990.append(nfa_data2000)).append(nfa_data2010))

# Dropping from country names.
nfa = nfa[nfa['country'] != 'World']

# Reseting index values.
nfa = nfa.set_index("country").reset_index()

In [3]:
nfa = nfa[(nfa['record'] == 'EFProdPerCap') | (nfa['record'] == 'EFConsPerCap')]
nfa.dropna(how='any',inplace=True)
nfa = nfa.set_index("country").reset_index()

In [4]:
nfa

,country,year,carbon,record
0,Afghanistan,1990,0.118341,EFConsPerCap
1,Afghanistan,1990,0.072586,EFProdPerCap
2,Albania,1990,0.781991,EFConsPerCap
3,Albania,1990,0.652289,EFProdPerCap
4,Angola,1990,0.134989,EFConsPerCap
5,Angola,1990,0.111797,EFProdPerCap
6,Argentina,1990,0.785846,EFConsPerCap
7,Argentina,1990,1.064232,EFProdPerCap
8,Australia,1990,4.328901,EFConsPerCap
9,Australia,1990,5.392423,EFProdPerCap


In [5]:
# Reformats NFA data.
i = 0
country_col = []
year_col = []
c_cons_col = []
c_prods_col = []

for row in nfa['country']:
    if i < len(nfa['country']):
        country_col.append(nfa['country'][i])
        year_col.append(nfa['year'][i])
        c_cons_col.append(nfa['carbon'][i])
        i += 1
        c_prods_col.append(nfa['carbon'][i])
        i += 1
    
nfa = pd.DataFrame()
nfa['country'] = country_col
nfa['year'] = year_col
nfa['c_consumed'] = c_cons_col
nfa['c_produced'] = c_prods_col

# UN Data

In [6]:
# Selects only columns with valid, single year names.
selected_columns = []
for column in un:
    if 'Unnamed' not in column:
        if '-' not in column:
            selected_columns.append(column)

# Turns .. into NaN values. Drops rows with only NaN values.
un = un.replace("..","NaN") 
un = (un[selected_columns])
un.dropna(thresh = 9, inplace = True)

# Drops bottom rows of summary data.
un.drop(un.tail(36).index,inplace=True)

# Transforms data from horizontal to vertical.
un = pd.melt(un, id_vars = 'Country', value_vars = ['1990','2000','2010', '2014', '2015'])

# Renames columns.
un = un.rename(columns = {'Country':'country','variable':'year','value':'hdi'})

# Final Dataset

In [7]:
un['country'] = un['country'].astype('string')
nfa['country'] = nfa['country'].astype('string')
un['year'] = un['year'].astype('int64')
nfa['year'] = nfa['year'].astype('int64')

In [8]:
data = nfa.merge(un, on = ['country','year'])

In [9]:
data

,country,year,c_consumed,c_produced,hdi
0,Afghanistan,1990,0.118341,0.072586,0.302
1,Albania,1990,0.781991,0.652289,0.65
2,Angola,1990,0.134989,0.111797,NaN
3,Argentina,1990,0.785846,1.064232,0.718
4,Australia,1990,4.328901,5.392423,0.871
5,Austria,1990,3.209693,2.688340,0.803
6,Bahamas,1990,3.376212,2.588036,NaN
7,Bahrain,1990,7.947046,8.219086,0.749
8,Barbados,1990,2.428270,1.403260,0.732
9,Bangladesh,1990,0.062713,0.041888,0.394


In [10]:
data.to_csv('Final_Dataset.csv')